In [1]:
import pandas as pd
import geopandas as geopd
import pickle
import shap
%run all_sources_helper.ipynb
%run helper.ipynb
%run domain_helper.ipynb

In [2]:
with open('../map_bc/static/model/random_forest_model.pkl', 'rb') as file:
    rf_model = pickle.load(file)

In [3]:
dfhs = health()
dfws = water_sources()
dfsp = stationary_sources()
dfes = emissions()
dffu = fuels()
dfms = marginalization()

C:\Users\Shobhit Jagga\AppData\Local\Temp\ipykernel_11528\4062585730.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfws[chemical_columns] = dfws[chemical_columns].applymap(replace_concentration_values)


In [4]:
gdf = geopd.read_file("../dataset/Mexico/geometry/2023_1_00_MUN/2023_1_00_MUN.shp")
gdf['CVE_MUN'] = gdf['CVE_MUN'].astype(str).str.lstrip('0').replace('', '0').astype(int)
gdf['CVE_ENT'] = gdf['CVE_ENT'].astype(str).str.lstrip('0').replace('', '0').astype(int)
gdf['state_code'] = gdf['CVE_ENT']
gdf['municipality_code'] = gdf['CVE_MUN']
gdf = gdf[['state_code','municipality_code','NOMGEO']]

In [5]:
df = pd.merge(dfes, dfhs, on=['state_code','municipality_code','year'], how='outer')
df = pd.merge(df, dfws, on=['state_code','municipality_code'], how='outer')
df = pd.merge(df, dfsp, on=['state_code','municipality_code'], how='outer')
df = pd.merge(df, dffu, on=['state_code','municipality_code'], how='outer')
df = pd.merge(df, dfms, on=['state_code','municipality_code'], how='outer')
df = pd.merge(df, pd.DataFrame(gdf), on=['state_code','municipality_code'], how='left')
cancer_cols = ['support','social_assistance','outpatient_consultation','hospitalization','support_cumsum','social_assistance_cumsum','outpatient_consultation_cumsum','hospitalization_cumsum','risk_count','risk_concentration_sure','risk_concentration_possible','st_risk','concentration','electricity_fuel','firewood_fuel','oil_fuel','IMN_2020'] + st_activities
for col in cancer_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df[cancer_cols] = df[cancer_cols].fillna(0)
df.rename(columns=final_col_mapping, inplace=True)

In [6]:
display(df[(df['state_code']==3) & (df['municipality_code']==3)])

,state_code,municipality_code,year,emission_concentration,support,social_assistance,outpatient_consultation,hospitalization,support_count,social_assistance_count,outpatient_consultation_count,hospitalization_count,carcinogenic_water_sources_count,water_sources_carcinogens_1,water_sources_carcinogens_2B,dumps,biomass,carpentry,cement_gypsum,sugarcane_cultivation,chemical_manufacturing,pesticide_manufacturing,gas_stations,incinerator,sugar_mill,brick_factory,mining,paper_mill,solvent_paint,high_traffic_supply,high_traffic_alameda,high_traffic_station,stationary_risk_sources,electricity_fuel,firewood_fuel,oil_fuel,marginalization,NOMGEO
318,3,3,2004.0,0.000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,257.388778,La Paz
319,3,3,2005.0,0.000031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,257.388778,La Paz
320,3,3,2006.0,0.068575,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,257.388778,La Paz
321,3,3,2007.0,0.000025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,257.388778,La Paz
322,3,3,2008.0,0.000010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,257.388778,La Paz
323,3,3,2009.0,1259.908835,0.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,257.388778,La Paz
324,3,3,2010.0,1232.067060,0.0,0.0,12.0,4.0,0.0,0.0,16.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,257.388778,La Paz
325,3,3,2011.0,2541.374098,0.0,0.0,42.0,7.0,0.0,0.0,58.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,257.388778,La Paz
326,3,3,2012.0,4943.287229,0.0,9.0,63.0,9.0,0.0,9.0,121.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,257.388778,La Paz
327,3,3,2013.0,497.148498,0.0,0.0,1.0,0.0,0.0,9.0,122.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,257.388778,La Paz


In [7]:
# df_features = pd.read_csv('../map_bc/static/model/features.all.csv')
fcol = [
    "support_count", "social_assistance_count", "outpatient_consultation_count", "hospitalization_count",
    "carcinogenic_water_sources_count","water_sources_carcinogens_2B","water_sources_carcinogens_1",
    "emission_concentration","marginalization", 
    "electricity_fuel", "firewood_fuel", "oil_fuel",
    "brick_factory", "stationary_risk_sources",
]
df['risk_prediction'] = rf_model.predict(df[fcol])
df['year'] = df['year'].fillna(0).astype(int)
df_features = df
df_features.to_csv("../map_bc/static/model/features.all.csv", index=False)
# with open("../map_bc/static/model/features.all.csv", 'r') as f:
    # lines = [line for line in f if line.strip()]
# with open("../map_bc/static/model/features.all.csv", 'w') as f:
    # f.writelines(lines)

In [8]:
display(df_features['year'].value_counts())
print(df['year'].isna().sum())

year
2011    2454
2010    2454
2009    2454
2008    2454
2016    2454
2017    2454
2018    2454
2019    2454
2012    2454
2013    2454
2014    2454
2015    2454
2020    2454
2021    2454
2022    2454
2006     404
2005     376
2007     373
2004     317
0         21
Name: count, dtype: int64

0


In [9]:
explainer = shap.TreeExplainer(rf_model)

In [10]:
def compute_risk(df, year):
    dfc = df[ (df['year']==year) ][['state_code','municipality_code','risk_prediction']].copy()
    output_file_path = '../map_bc/static/risks/riskprofile.'+str(year)+'.csv'
    dfc.to_csv(output_file_path, index=False)
    with open(output_file_path, 'r') as f:
        lines = [line for line in f if line.strip()]
    with open(output_file_path, 'w') as f:
        f.writelines(lines)

In [11]:
# df_risk = pd.read_csv('../map_bc/static/risks/riskprofile.all.csv')
df_risk = df_features[['state_code','municipality_code','risk_prediction','year']]
display(df_risk)
years = df_risk['year'].unique()
for year in years:
    compute_risk(df_risk, year)

,state_code,municipality_code,risk_prediction,year
0,1,1,17.484338,2004
1,1,1,18.258158,2005
2,1,1,12.177382,2006
3,1,1,20.119159,2007
4,1,1,19.148807,2008
...,...,...,...,...
38296,32,58,0.828050,2018
38297,32,58,0.828050,2019
38298,32,58,0.828050,2020
38299,32,58,0.828050,2021


In [12]:
def compute_contribution(df, model, state_code, municipality_code, year):
    dfc = df[ (df['state_code']==state_code) & (df['municipality_code']==municipality_code) & (df['year']==year)][fcol].copy()
    # display(dfc.shape)
    shap_values = explainer.shap_values(dfc)
    expected_value = explainer.expected_value
    temp_df = pd.DataFrame({
        'Source': fcol,
        'Contribution': shap_values[0],
    })
    temp_df = temp_df.reindex(temp_df['Contribution'].abs().sort_values(ascending=False).index)
    expected_value = explainer.expected_value
    expected_value = expected_value if isinstance(expected_value, (int, float)) else expected_value[0]
    prediction = rf_model.predict(dfc)
    pred_df = pd.DataFrame({'Source': ['Prediction'], 'Contribution': prediction})
    cont_df = pd.concat([pred_df, temp_df], ignore_index=True)
    cont_df['Contribution'] = cont_df['Contribution'].round(1)
    output_file_path = '../map_bc/static/risks/riskprofile.'+str(state_code)+'.'+str(municipality_code)+'.'+str(year)+'.csv'
    cont_df.to_csv(output_file_path, index=False)
    with open(output_file_path, 'r') as f:
        lines = [line for line in f if line.strip()]
    with open(output_file_path, 'w') as f:
        f.writelines(lines)
    # display(cont_df)

In [13]:
for index, row in df_features.iterrows():
    # display(year)
    year = int(row['year'])
    if year!=2022:
        continue
    state_code = int(row['state_code'])
    mun_code = int(row['municipality_code'])
    # display(state_code)
    # display(mun_code)
    compute_contribution(df_features, rf_model, state_code, mun_code, year)
# compute_contribution(df_features, rf_model, 1, 1)